## [CSI-Workflow] Global Shelf--Shelf-Width Transects 
<br>
Notebook Scope: <br>
This document describes the overall workflow associated with capturing shelf widths, locating the shelf-break along transects (the point where the transect intersects the shelf break), and generating a set of descriptive statistics and graphic analysis products to help to tailor and bolter our hypotheses.

- Create the Width Transects from the Profiles
- Create the shelf break intersection points
- Create the shelf break intersection sample buffers (5km dia)
- Sample the slope and depth shelf models using the sample buffers
- Export the results to an ESRI Shapefile and a comma-separated values text file

#### The Working Zones

|1   | Eastern North America <br> 
|2	 | Western North America <br>
|3	 | Eastern South America <br>
|4	 | Western South America <br>
|5	 | Europe <br>
|6	 | Western Africa <br>
|7	 | Indian/Oceana <br>
|8	 | Eastern Asia <br>
|9	 | Australia <br>
|10	 | Arctic <br>
|11	 | Antarctic <br>
|12	 | Mediterranean <br> 
|13	 | New Zealand (not currently included in the analysis)<br>

#### WORKFLOW for capturing the overall shelf width and various descriptive statistics at the shelf break via the Cross-Shelf Transect Method

1.) [in QGIS] Intersect Profile Transects w/the Shelf 
		- input Profile Transects
		- intersect Shelf Geomorphon Layer (vector)
		- output temp transects  (overwrite, as necessary and delete in the end)
		
2.) [in QGIS] Digitize new Shelf Width Transects using the temp transects as snapping guides.
		- calc id field = rownumbers
		- add new field TotalWidth (int 10) and pop. with Geometry->$length
		- output: Shelf Width Transects
		

3.) [in QGIS] Generate from the new Shelf Width Transects nodes for the start and end of each transect. Edit to delete all the landward points (nodes) leaving behind only those points that occur along the shelf break
		- output: Shelf_Break_Sample_Points
		
4.) [in QGIS] Generate a 5 km buffer (buffer distance 2500 m) around each sample point
		- output Shelf Break Sample Areas
		
        
5.) [in GRASS] Import the Shelf Break Sample Areas vector layer in to GRASS and:

	a.) interrogate the shelf bathymetry and slopes using the shelf break sample areas and v.rast.stats:


	b.) export the updated (the attribute table will be updated with the above stats) Shelf Break Sample Areas vector layer out, overwriting the existing shape file as you go. 
	
	c.) export the updated attribute table from the Shelf Break Sample Areas w/o geometry  as a comma-separated value file (*.csv):

db.out.ogr --o input=Arctic_ShelfBreak_SampleAreas_Zone10 output=/Users/paulp/Documents/csi/cshelves/data/vector/shelves/ShelfBreakSamples/Arctic_ShelfBreak_SampleAreas_Zone10 format=CSV table=Arctic_ShelfBreak_SampleAreas_Zone10



In [1]:
import os, sys

# set the location and mapset for the GRASS instance to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'csi_shelf_mapping_global'                       # name of location
mapset = 'user'                     # name of mapset containing data

In [2]:
# This class definition is used to initialize an instance of GRASS 7.x in this domain space
class GrassBASE:
    def initGRASSEnv( self, database, loc, map ):
        gisbase=os.environ[ 'GISBASE' ] = '/Applications/GRASS-7.1.app/Contents/MacOS'
        gisdbase=database
        location=loc
        mapset=map
        sys.path.append(os.path.join(os.environ['GISBASE'], "etc", "python"))
        import grass.script as gscript
        import grass.script.setup as gsetup
        gsetup.init( gisbase, gisdbase, location, mapset )
        print(gscript.gisenv())
        print('GRASS Application base path:',gisbase)

In [4]:
# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'csi_shelf_mapping_global'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib


#### Load the grass scripting module, and set the file paths, zones, and names, and finally, the working region (for all zones save for the Arctic and Antarctic)

In [5]:
### set file paths and region

import grass.script as gscript


r='ETOPO1_World_1km'  # region

f_src_path='/Users/paulp/Documents/csi/cshelves/data/vector/shelves/ShelfbyZone/Zone'
f_out_path='/Users/paulp/Documents/csi/cshelves/data/vector/shelves/ShelfBreakSamples/'

zone_list=['1','2','3','4','5','6','7','8','9','12']
areas_list=['ECoastNAmer_Shelf_Break_Sample_Areas_Zone1','WCoastNAmer_Shelf_Break_Sample_Areas_Zone2', \
           'ECoastSAmer_Shelf_Break_Sample_Areas_Zone3','WCoastSAmer_Shelf_Break_Sample_Areas_Zone4',
           'European_Shelf_Break_Sample_Areas_Zone5','WCoastAfrica_Shelf_Break_Sample_Areas_Zone6', \
           'IndianOceana_Shelf_Break_Sample_Areas_Zone7','ECoastAsia_Shelf_Break_Sample_Areas_Zone8',\
            'Australia_Shelf_Break_Sample_Areas_Zone9','Mediterranean_Shelf_Break_Sample_Areas_Zone12']

###'Arctic_Shelf_Break_Sample_Areas_Zone10', 'Antarctic_Shelf_Break_Sample_Areas_Zone11',

gscript.run_command('g.region', region=r)

0

#### Import the sample area polygons for each zone into GRASS


Notes:
- if the file to be import is already present in GRASS the existing will be overwritten by the new

In [49]:
# ## import the shelf-break sampling areas (5km dia buffers) for each zone into GRASS
import grass.script as gscript

for zone, area in zip(zone_list, areas_list):
    current_zone=f_src_path+zone+'/'+area
    gscript.run_command('v.in.ogr', flags='o', overwrite=True, input=current_zone+'.shp', output=area)

#### Capture samples from bathy and slope DSMs from within each sample areas (using v.rast.stats)

In [6]:
# ## Use v.rast..stats to sample depths and slopes in those sampling areas. 

import grass.script as gscript

stats='average,stddev,median'

gscript.run_command('g.region', flags='p', region='ETOPO1_World_1km')

for area in (areas_list):

    # ## depth samplng
    gscript.run_command('v.rast.stats', flags='c', overwrite=True, map=area, raster='ETOPO1_World_Shelf_bathy_1km', \
                        method=stats, column_prefix='z')
  
    # ## slope sampling
    gscript.run_command('v.rast.stats', flags='c', overwrite=True, map=area,  raster='ETOPO1_World_Shelf_Slope_1km', \
                        method=stats, column_prefix='m')

#### Export the feature attribute data contents for each zone sample areas layer from GRASS for outside processing into numeric and graphic analysis products (e.g., box plots, pie charts)

In [9]:

import grass.script as gscript

for area in(areas_list):
    gscript.run_command('db.out.ogr', overwrite=True, input=area, output=f_out_path+area, format='CSV' )


#### Export the feature geometry and descriptive attribute contents for each zone sample areas layer from GRASS for outside analysis products.

In [10]:
import grass.script as gscript

for area in (areas_list):
    gscript.run_command('v.out.ogr', overwrite=True, input=area, output=f_out_path+area, format='ESRI_Shapefile')

### Now, let's do the poles...

#### first, the Arctic:

In [21]:
# set the location and mapset for the GRASS instance (for the Arctic) to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'CSM_Arctic_Region              '                       # name of location
mapset = 'user'  

# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'CSM_Arctic_Region'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib


In [22]:
### set file paths and region

import grass.script as gscript

stats='average,stddev,median'

gscript.run_command('g.region', region='ETOPO1_Arctic_1km')  # set the region

src_file='/Users/paulp/Documents/csi/cshelves/data/vector/shelves/ShelfbyZone/Zone10/Arctic_Shelf_Break_Sample_Areas_Zone10.shp'
out_file='/Users/paulp/Documents/csi/cshelves/data/vector/shelves/ShelfBreakSamples/Arctic_Shelf_Break_Sample_Areas_Zone10'

#gscript.run_command('v.in.ogr', flags='o', overwrite=True, input=src_file, output='Arctic_Shelf_Break_Sample_Areas_Zone10)

# ## depth samplng
gscript.run_command('v.rast.stats', flags='c', overwrite=True, map='Arctic_Shelf_Break_Sample_Areas_Zone10', \
                    raster='ETOPO1_Arctic_Shelf_bathy_1km', method=stats,column_prefix='z')
  
# ## slope sampling
gscript.run_command('v.rast.stats', flags='c', overwrite=True, map='Arctic_Shelf_Break_Sample_Areas_Zone10', \
                    raster='ETOPO1_Arctic_Shelf_slope_1km', method=stats,column_prefix='m')

# ## export feature attribute table as ASCII.csv
gscript.run_command('db.out.ogr', overwrite=True, input='Arctic_Shelf_Break_Sample_Areas_Zone10', output=out_file, \
                    format='CSV' )


0

In [23]:
# ## export geometry and feature attributes to ESRI Shapefile    
gscript.run_command('v.out.ogr', overwrite=True, input='Arctic_Shelf_Break_Sample_Areas_Zone10', \
                    output='/Users/paulp/Documents/csi/cshelves/data/vector/shelves', format='ESRI_Shapefile')

0

#### then, the Antarctic...

In [27]:
# set the location and mapset for the GRASS instance (for the Arctic) to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'CSM_Antarctic_Region              '                       # name of location
mapset = 'user'  

# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'CSM_Antarctic_Region'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib


In [28]:
### set file paths and region

import grass.script as gscript

stats='average,stddev,median'

gscript.run_command('g.region', region='ETOPO1_antarctic_1km')  # set the region

src_file='/Users/paulp/Documents/csi/cshelves/data/vector/shelves/ShelfbyZone/Zone11/Antarctic_Shelf_Break_Sample_Areas_Zone11.shp'
out_path='/Users/paulp/Documents/csi/cshelves/data/vector/shelves/ShelfBreakSamples/'

gscript.run_command('v.in.ogr', flags='o', overwrite=True, input=src_file, output='Antarctic_Shelf_Break_Sample_Areas_Zone11')

# ## depth samplng
gscript.run_command('v.rast.stats', flags='c', overwrite=True, map='Antarctic_Shelf_Break_Sample_Areas_Zone11', \
                    raster='ETOPO1_Antarctic_Shelf_bathy_1km', method=stats,column_prefix='z')
  
# ## slope sampling
gscript.run_command('v.rast.stats', flags='c', overwrite=True, map='Antarctic_Shelf_Break_Sample_Areas_Zone11', \
                    raster='ETOPO1_Antarctic_Shelf_slope_1km', method=stats, column_prefix='m')


# ## export feature attribute table as ASCII.csv
gscript.run_command('db.out.ogr', overwrite=True, input='Antarctic_Shelf_Break_Sample_Areas_Zone11', \
                    output=out_path+'Antarctic_Shelf_Break_Sample_Areas_Zone11', format='CSV' )


0

In [29]:
# ## export geometry and feature attributes to ESRI Shapefile  

import grass.script as gscript

gscript.run_command('v.out.ogr', flags='u', overwrite=True, input='Antarctic_Shelf_Break_Sample_Areas_Zone11', \
                    output='/Users/paulp/Documents/csi/cshelves/data/vector/shelves', format='ESRI_Shapefile')

0